In [4]:
# Import Library
import os
import time
import numpy as np
import cv2
import glob
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler,scale
from sklearn.model_selection import train_test_split, GridSearchCV
from skimage.feature import hog

In [5]:
######################
# Get Color Features #
######################
def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                        
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [6]:
####################
# Get Hog Features #
####################
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, 
                                  orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, 
                                  feature_vector=feature_vec, 
                                  block_norm="L2-Hys")
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, 
                       orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, 
                       feature_vector=feature_vec, 
                       block_norm="L2-Hys")
        return features

In [7]:
# Define a function to extract features from a single image window
# just for a single image rather than list of images
def single_img_features(img, 
                        color_space='RGB',
                        spatial_size=(32, 32),
                        hist_bins=32,
                        orient=9, 
                        pix_per_cell=8, 
                        cell_per_block=2, 
                        hog_channel=0,
                        spatial_feat=True, 
                        hist_feat=True, 
                        hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
        elif color_space == 'GRAY':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        hog_features = []
        if color_space == 'GRAY':
            hog_features = get_hog_features(feature_image, 
                                            orient, 
                                            pix_per_cell,
                                            cell_per_block, 
                                            vis=False, 
                                            feature_vec=True)
        elif hog_channel == 'ALL':
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                                     orient, 
                                                     pix_per_cell, 
                                                     cell_per_block, 
                                                     vis=False, 
                                                     feature_vec=True))
            hog_features = np.ravel(hog_features)
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel],
                                            orient, 
                                            pix_per_cell, 
                                            cell_per_block, 
                                            vis=False, 
                                            feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs,
                     color_space='RGB',
                     spatial_size=(32, 32),
                     hist_bins=32, 
                     orient=9, 
                     pix_per_cell=8,
                     cell_per_block=2, 
                     hog_channel=0,
                     spatial_feat=True,
                     hist_feat=True, 
                     hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
#         img = mpimg.imread(file)
        img = cv2.imread(file)
        img_features = single_img_features(img,
                                           color_space=color_space, 
                                           spatial_size=spatial_size, 
                                           hist_bins=hist_bins, 
                                           orient=orient, 
                                           pix_per_cell=pix_per_cell, 
                                           cell_per_block=cell_per_block, 
                                           hog_channel=hog_channel,
                                           spatial_feat=spatial_feat, 
                                           hist_feat=hist_feat, 
                                           hog_feat=hog_feat)
        features.append(img_features)
    # Return list of feature vectors
    return features

In [11]:
arr_color_space = ['HSV', 'HLS', 'GRAY']
arr_orient = [8,9]
# arr_spatial_feat = [True, False]
# arr_hist_feat = [True, False]
# arr_hog_feat = [True, False]

In [12]:
#######################
# Classifier Training #
#######################
color_space = 'HLS' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb, GRAY
orient = 8  # HOG orientations
pix_per_cell = 16 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
    
images_cars = glob.glob('./data/vehicles/*/*')
images_notcars = glob.glob('./data/non-vehicles/*/*')
list_cars = []
list_notcars = []
for image in images_cars:
    list_cars.append(image)
for image in images_notcars:
    list_notcars.append(image)

data_info = {}
data_info["n_cars"] = len(list_cars)
data_info["n_notcars"] = len(list_notcars)
example_img = cv2.imread(list_cars[0])
data_info["image_shape"] = example_img.shape
data_info["data_type"] = example_img.dtype

print('cars:',data_info["n_cars"],', notcars:', data_info["n_notcars"], ', shape:', data_info["image_shape"], data_info["data_type"])


def train():
    
#     aug_cars = glob.glob('./data/aug/cars/*.jpg')
#     aug_notcars = glob.glob('./data/aug/notcars*/*.jpg')
#     for image in aug_cars:
#         list_cars.append(image)
#     for image in aug_notcars:
#         list_notcars.append(image)
    print(color_space, pix_per_cell, cell_per_block, orient, hog_channel, spatial_feat, hist_feat, hog_feat)
    
    t = time.time()
    car_features = []
    notcar_features = []
    car_features = extract_features(list_cars,
                                    color_space=color_space, 
                                    spatial_size=spatial_size, 
                                    hist_bins=hist_bins, 
                                    orient=orient, 
                                    pix_per_cell=pix_per_cell, 
                                    cell_per_block=cell_per_block, 
                                    hog_channel=hog_channel, 
                                    spatial_feat=spatial_feat, 
                                    hist_feat=hist_feat,
                                    hog_feat=hog_feat)
    notcar_features = extract_features(list_notcars, 
                                       color_space=color_space, 
                                       spatial_size=spatial_size, 
                                       hist_bins=hist_bins, 
                                       orient=orient, 
                                       pix_per_cell=pix_per_cell, 
                                       cell_per_block=cell_per_block, 
                                       hog_channel=hog_channel, 
                                       spatial_feat=spatial_feat, 
                                       hist_feat=hist_feat, 
                                       hog_feat=hog_feat)
    t2 = time.time()
    print(round(t2-t, 5), 'Seconds to extract features...')
    
    X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)
    print(X.shape, scaled_X.shape)
    print(len(car_features), len(notcar_features))

    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=50)
    print('Feature vector length:', len(X_train[0]))
    

    clf = LinearSVC()
#     clf = SVC()
#     svr = SVC()
#     parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]} 
#     clf = GridSearchCV(svr, parameters)
    
    # Check the training time for the SVC
    t = time.time()
    clf.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2-t, 2), 'Seconds to train SVC...')
#     print('SVC best params:', clf.best_params_)
    
    # Check the score of the SVC
    score = round(clf.score(X_test, y_test), 4)
    print('Test Accuracy of SVC = ', score)
    
#     # Check the prediction time for a single sample
#     t = time.time()
#     n_predict = 10
#     print('SVC predicts: ', clf.predict(X_test[0:n_predict]))
#     print('SVC   labels: ', y_test[0:n_predict])
#     t2 = time.time()
#     print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')
    
    # Save the model
    data_model = {}
    data_model['clf'] = clf
    data_model['X_scaler'] = X_scaler
    data_model['pix_per_cell'] = pix_per_cell
    data_model['cell_per_block'] = cell_per_block
    data_model['orient'] = orient
    data_model['color_space'] = color_space
    data_model['hog_channel'] = hog_channel
    data_model['spatial_size'] = spatial_size
    data_model['hist_bins'] = hist_bins
    data_model['spatial_feat'] = spatial_feat
    data_model['hist_feat'] = hist_feat
    data_model['hog_feat'] = hog_feat
    
    with open('./svc_{}_{}_{}_{}_{}_{}_{}_{}_{}.p'.format(score, color_space, pix_per_cell, cell_per_block, orient, hog_channel, spatial_feat, hist_feat, hog_feat), 'wb') as file:
        pickle.dump(data_model, file, True)
    print('SVC Saved...')
    
    return clf

#########
# Train #
#########


# for param1 in arr_color_space:
#     for param2 in arr_orient:
#         for param3 in arr_cell_per_block:
#             for param4 in arr_hog_channel:
#                 color_space = param1
#                 orient = param2
#                 cell_per_block = param3
#                 hog_channel = param4
                
#                 train()

for param1 in arr_color_space:
    for param2 in arr_orient:
        color_space = param1
        orient = param2

        train()

# X_train, X_test, y_train, y_test, hog_params = preprocess()
# clf = train(X_train, X_test, y_train, y_test, hog_params)

cars: 8792 , notcars: 8968 , shape: (64, 64, 3) uint8
HSV 16 2 8 ALL True True True
57.34916 Seconds to extract features...
(17760, 1728) (17760, 1728)
8792 8968
Feature vector length: 1728
3.78 Seconds to train SVC...
Test Accuracy of SVC =  0.9845
SVC Saved...
HSV 16 2 9 ALL True True True
67.51795 Seconds to extract features...
(17760, 1836) (17760, 1836)
8792 8968
Feature vector length: 1836
4.21 Seconds to train SVC...
Test Accuracy of SVC =  0.9845
SVC Saved...
HLS 16 2 8 ALL True True True
69.64175 Seconds to extract features...
(17760, 1728) (17760, 1728)
8792 8968
Feature vector length: 1728
4.19 Seconds to train SVC...
Test Accuracy of SVC =  0.9794
SVC Saved...
HLS 16 2 9 ALL True True True
68.3625 Seconds to extract features...
(17760, 1836) (17760, 1836)
8792 8968
Feature vector length: 1836
4.52 Seconds to train SVC...
Test Accuracy of SVC =  0.9792
SVC Saved...
GRAY 16 2 8 ALL True True True


IndexError: too many indices for array